In [1]:
%load_ext autoreload
%autoreload 2

import os, sys
import tensorflow as tf
import cv2
import numpy as np
from matplotlib import pyplot as plt
import seaborn as sns
sys.path.append("/home/files/vision_tree")
from utils.session_config import setup_gpus

In [2]:
os.environ["CUDA_VISIBLE_DEVICES"]="1"
setup_gpus(memory_fraction=0.2)

1 Physical GPUs, 1 Logical GPUs


In [3]:
from tensorflow.keras.layers import Conv2D, MaxPool2D, Flatten, Dense, ReLU, BatchNormalization, GlobalAveragePooling2D
from utils.losses import L2_Orthogonal

In [4]:
data_0 = np.load("/home/files/datasets/mnist/0.npy")
label_0 = np.zeros(len(data_0), np.int32)
data_5 = np.load("/home/files/datasets/mnist/5.npy")
label_5 = np.ones(len(data_5), np.int32)
data_7 = np.load("/home/files/datasets/mnist/7.npy")
label_7 = np.ones(len(data_7), np.int32)*2

train_x = tf.concat([data_0, data_5, data_7], axis=0)
train_x = tf.cast(train_x, tf.float32)
train_x = train_x[:,:,:,tf.newaxis]/255.
train_y = tf.concat([label_0, label_5, label_7], axis=0)

x = tf.data.Dataset.from_tensor_slices(train_x)
y = tf.data.Dataset.from_tensor_slices(train_y)
ds = tf.data.Dataset.zip((x, y))
ds = ds.shuffle(len(train_x), reshuffle_each_iteration=False).shuffle(1000).batch(32).prefetch(tf.data.experimental.AUTOTUNE)

model = tf.keras.Sequential()
model.add(Conv2D(16, (3,3), kernel_regularizer=L2_Orthogonal()))
model.add(ReLU())
model.add(MaxPool2D())
model.add(Conv2D(16, (3,3), kernel_regularizer=L2_Orthogonal()))
model.add(ReLU())
model.add(MaxPool2D())
model.add(Conv2D(16, (3,3), kernel_regularizer=L2_Orthogonal()))
model.add(ReLU())
model.add(GlobalAveragePooling2D())
model.add(Dense(3, activation='softmax', kernel_regularizer='l2'))

model.compile(optimizer='Adam', loss=tf.keras.losses.SparseCategoricalCrossentropy(), metrics=['acc'])
model.fit(ds, epochs=5)

Epoch 1/5
551/551 [==============================] - 2s 4ms/step - loss: 0.3012 - acc: 0.9349
Epoch 2/5
551/551 [==============================] - 2s 4ms/step - loss: 0.1208 - acc: 0.9899
Epoch 3/5
551/551 [==============================] - 2s 4ms/step - loss: 0.1074 - acc: 0.9924
Epoch 4/5
551/551 [==============================] - 2s 4ms/step - loss: 0.1011 - acc: 0.9920
Epoch 5/5
551/551 [==============================] - 2s 4ms/step - loss: 0.0940 - acc: 0.9936


NameError: name 'test_x' is not defined

In [5]:
model.save("ckpt/mnist_cnn/orthogonal_057.h5")

In [6]:

model = tf.keras.Sequential()
model.add(Conv2D(16, (3,3)))
model.add(ReLU())
model.add(MaxPool2D())
model.add(Conv2D(16, (3,3)))
model.add(ReLU())
model.add(MaxPool2D())
model.add(Conv2D(16, (3,3)))
model.add(ReLU())
model.add(GlobalAveragePooling2D())
model.add(Dense(3, activation='softmax'))

model.compile(optimizer='Adam', loss=tf.keras.losses.SparseCategoricalCrossentropy(), metrics=['acc'])
model.fit(ds, epochs=5)

Epoch 1/5
551/551 [==============================] - 1s 2ms/step - loss: 0.2094 - acc: 0.9347
Epoch 2/5
551/551 [==============================] - 1s 2ms/step - loss: 0.0371 - acc: 0.9886
Epoch 3/5
551/551 [==============================] - 1s 2ms/step - loss: 0.0261 - acc: 0.9920
Epoch 4/5
551/551 [==============================] - 1s 2ms/step - loss: 0.0197 - acc: 0.9934
Epoch 5/5
551/551 [==============================] - 1s 2ms/step - loss: 0.0172 - acc: 0.9945


In [7]:
model.save("ckpt/mnist_cnn/057.h5")